# List of postal codes of Canada
## How to get the content from a website and deal with it

### First, import librairies

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Get the content from the website

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
soup = BeautifulSoup(requests.get(url).text)

### Extract the content of the table and parse in dataframe

In [3]:
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]
print('number of rows of the dataframe',df.shape)

number of rows of the dataframe (289, 3)


### Reset default indexes and define columns names

In [4]:
df.reset_index(drop=True,inplace=True)
df.columns = df.iloc[0].tolist()
df.drop(0,inplace=True)

 ### Drop rows with borough not assigned

In [5]:
df = df[df.Borough != 'Not assigned']

In [6]:
new_data = []
for i,row in df.iterrows():
    if(len(df.loc[df.Postcode == row.Postcode]) >1):
        postal_code = row.Postcode
        borough = row.Borough
        list_neighborhood = []
        for data in df.loc[df.Postcode == row.Postcode]['Neighbourhood']:
            list_neighborhood.append(data)
            new_neighborhood = ', '.join(list_neighborhood)
        df = df[df.Postcode != row.Postcode]
        new_data.append([postal_code,borough,new_neighborhood])

### Check rows to get rows with Neighborhood not assigned in order to assign it with the same bourough value

In [7]:
for i,row in df.iterrows():
   if(row.Neighbourhood == 'Not assigned'):
    df.loc[i]['Neighbourhood'] = df.loc[i]['Borough'] 

### Concat the two dataframe et display the tail of the dataframe

In [8]:
df_can = pd.DataFrame(new_data,columns=['Postcode','Borough','Neighbourhood'])

In [9]:
df = df.append(df_can)
df.tail()

,Postcode,Borough,Neighbourhood
52,M4X,Downtown Toronto,"Cabbagetown, St. James Town"
53,M5X,Downtown Toronto,"First Canadian Place, Underground city"
54,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
55,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
56,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."


### Displat the shape of final the dataframe

In [10]:
print('number of rows of the dataframe : ',df.shape)

number of rows of the dataframe :  (103, 3)


### Save the dataframe into a csv file

In [17]:
df.to_csv('postcode_canada.csv',index=False)